### This program needs a fix on creation of mini-batches at its end

In [4]:
import numpy as np
import pandas as pd
import os
import re

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
count=CountVectorizer()
docs = np.array(['The sun is shining on Zhongping',
                 'The weather is sweet',
                 'The sun is shining, the weather is sweet,'
                 'and one and one is two'])
bag = count.fit_transform(docs)

print(count.vocabulary_)
print(bag.toarray())
print(count.vocabulary_.get('zhongping'))

{'the': 7, 'sun': 5, 'is': 1, 'shining': 4, 'on': 2, 'zhongping': 10, 'weather': 9, 'sweet': 6, 'and': 0, 'one': 3, 'two': 8}
[[0 1 1 0 1 1 0 1 0 0 1]
 [0 1 0 0 0 0 1 1 0 1 0]
 [2 3 0 2 1 1 1 2 1 1 0]]
10


In [7]:
count.get_feature_names()

['and',
 'is',
 'on',
 'one',
 'shining',
 'sun',
 'sweet',
 'the',
 'two',
 'weather',
 'zhongping']

In [8]:
rNews=pd.read_csv('RedditNews.csv', encoding='latin-1')
dNews=pd.read_csv('Combined_News_DJIA.csv', encoding='latin-1')

In [9]:
def preprocessor(text):
    text = (re.sub('[\W]+',' ', text.lower() + ''))
    text = re.sub("b ",' ',text)
    return text

In [10]:
dNews['Top1'] = dNews['Top1'].apply(preprocessor)
dNews['Top2'] = dNews['Top2'].apply(preprocessor)
dNews['Top3'] = dNews['Top3'].apply(preprocessor)

In [11]:
#def tokenizer(text):
#    return text.split()
#tokenizer(dNews['Top1'][0])

In [12]:
dNews.shape

(1989, 27)

In [13]:
# X_train = dNews.loc[:1000,"Top1"].values
X1 = dNews.loc[:,"Top1"].values
y1 = dNews.loc[:,"Label"].values
X2 = dNews.loc[:,"Top2"].values
y2 = dNews.loc[:,"Label"].values
X3 = dNews.loc[:,"Top3"].values
y3 = dNews.loc[:,"Label"].values
X = np.concatenate((X1, X2, X3))
y = np.concatenate((y1, y2, y3))
print(X.shape)
print(y.shape)

(5967,)
(5967,)


In [14]:
vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(2,2))
#vectorizer = CountVectorizer(ngram_range=(2,2))

In [15]:
# X_fitted=vectorizer.fit_transform(X)
# X_fitted.shape

In [16]:
# print(X_fitted)

In [17]:
# print(vectorizer.vocabulary_)
# print(vectorizer.vocabulary_.get('war'))

In [18]:
# offset=1
# split_index = 1800
# X_train = X_fitted[:split_index]
# X_test = X_fitted[split_index:]
# y_train= y[:split_index]
# y_test = y[split_index:]

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=42)
#.   
X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)

In [20]:
# Concatenate the "Date column" to vectorized X-Train.
# Batch the data for before & After crash...
#. How do we itroduce a "line of news" and see how the model predicts it?


In [21]:
X_train.shape

(2983, 29904)

In [22]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

/Users/piruzalemi/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.5197721179624665

In [23]:
vec_words = vectorizer.get_feature_names()
model_coef = clf.coef_.tolist()[0]
coef_df = pd.DataFrame({'word': vec_words,
                        'coef': model_coef})
top_coef_df = coef_df.sort_values(['coef', 'word'], ascending=[0, 1])
#top_coef_df.head(10)
top_coef_df.tail(10)
#top_coef_df

,word,coef
23414,saudi prince,-0.449656
15086,laden dead,-0.459273
29690,year low,-0.464197
29543,world war,-0.480403
12656,hong kong,-0.494691
11013,gang raped,-0.496560
19490,passes law,-0.497646
12002,hacked death,-0.503123
16782,medical marijuana,-0.528033
4878,catholic church,-0.728149


In [24]:
# Load the library with the CountVectorizer method
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [25]:
from sklearn.metrics import confusion_matrix
y_true = [2, 0, 2, 2, 0, 1]
y_pred = [0, 0, 2, 2, 0, 2]
confusion_matrix(y_true, y_pred)

array([[2, 0, 0],
       [0, 0, 1],
       [1, 0, 2]])

## Naive Bayes

In [26]:
from sklearn.naive_bayes import BernoulliNB
# from sklearn.feature_extraction.text import CountVectorizer

# vectorizer = CountVectorizer()
naive_bayes = BernoulliNB()
naive_bayes.fit(X_train, y_train)
naive_bayes.score(X_test, y_test)


0.5304959785522788

In [27]:
naive_bayes.score(X_train, y_train)

0.9544083137780758

In [28]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((2983, 29904), (2983,), (2984, 29904), (2984,))

In [29]:
#.  Add RF + Dates.....(Laging time) ..... Day 1... Day 500 Transform the dates...
#.  10 sentences + 50 unique words + Date...dot.toarray 

# Random Forest Model

In [30]:
from sklearn.ensemble import RandomForestRegressor
#from sklearn.ensemble import RandomForestRegressor

In [31]:
forest = RandomForestRegressor(n_estimators=50, criterion = 'mse', random_state=1, n_jobs=-1)


In [32]:
forest.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
                      oob_score=False, random_state=1, verbose=0,
                      warm_start=False)

In [33]:
y_train_pred = forest.predict(X_train)
y_train_pred

array([0.32, 1.  , 0.22, ..., 0.88, 1.  , 0.4 ])

In [34]:
y_test_pred=forest.predict(X_test)
y_test_pred

array([0.42, 0.7 , 1.  , ..., 0.64, 1.  , 0.98])

In [35]:
y_test_pred.shape

(2984,)

In [36]:
y_train_pred.shape

(2983,)

In [37]:
import matplotlib.pyplot as plt

In [38]:
import seaborn as sns

In [39]:
cols = X_train[-1][-1]
cols.shape

(1, 29904)

In [40]:
#sns.pairplot([cols], size = 2.5)

In [41]:
#confusion_matrix(X_train_pred, X_train)

In [42]:
from sklearn.metrics import mean_squared_error

In [43]:

print('MSE train: %.3f, test: %.3f' % (
    mean_squared_error(y_train, y_train_pred),
    mean_squared_error(y_test,y_test_pred)))

MSE train: 0.058, test: 0.417


In [44]:
from sklearn.metrics import r2_score

In [45]:
print('R^2 train: %.3f, test: %.3f' %(
    r2_score(y_train, y_train_pred),
    r2_score(y_test, y_test_pred)))

R^2 train: 0.765, test: -0.673


In [46]:
#target = [y_test]
#target_names = ["negative", "positive"]

In [47]:
# Calculate classification report
# from sklearn.metrics import classification_report
# predictions = forest.predict(X_test)
# print(classification_report(y_test, predictions,
#                            target_names=target_names))

## Latent Dirichlet Allocation
### Assign Topics to unlabelled text dopcuments

In [48]:
# Note we set the maximum document frequency to be considered to 10%, to exclude words that occur 
#. too frequently across documents.
#
# The rational behind the removal of frequency occuring word  is that these might be common words occuring across
# all documents. Also we limit the the number of words to be considered, to the most frequently occuring 200 words
# max_features=200
count2 = CountVectorizer(stop_words='english', max_df=0.1,max_features=200)

In [49]:
X=count2.fit_transform(dNews['Top1'].values)

In [50]:
from sklearn.decomposition import LatentDirichletAllocation

In [51]:
# n_topics did not work and i changed it to n_components
lda = LatentDirichletAllocation(n_components=10, random_state=123,learning_method='batch')

In [52]:
X_topics=lda.fit_transform(X)

In [53]:
lda.components_.shape

(10, 200)

In [54]:
n_top_words = 10

In [55]:
feature_names = count2.get_feature_names()
#feature_names

In [56]:
for topic_idx, topic in enumerate(lda.components_):
    print("Topic %d:" % (topic_idx + 1))
    print(" | ".join([feature_names[i]
                   for i in topic.argsort()\
                       [:-n_top_words -1:-1]]))

Topic 1:
new | france | australia | free | cancer | drugs | protest | days | government | dutch
Topic 2:
uk | north | korea | million | just | south | dead | iran | nuclear | city
Topic 3:
women | woman | death | ban | isis | india | police | country | man | food
Topic 4:
year | china | police | old | saudi | canada | british | girl | arabia | russia
Topic 5:
people | law | president | right | united | protesters | canadian | know | marijuana | egypt
Topic 6:
wikileaks | state | money | bans | bank | billion | oil | israeli | secret | court
Topic 7:
000 | snowden | internet | power | nsa | data | children | young | german | 10
Topic 8:
world | news | government | military | london | calls | american | war | says | 20
Topic 9:
says | israel | human | rights | russian | party | european | pirate | palestinian | europe
Topic 10:
years | drug | killed | minister | chinese | mexico | life | pope | prime | 000


In [59]:
#.  Note the index = 15 for Topic #16 (Alemi)
Topic9 =X_topics[:, 9].argsort()[::-1]

In [60]:
# Based on the most importantyv words for each topic, we may guess that Latent Dirichlet Allocation (LDA)
# puts them in the above 20 topics. I here print  say Topic 9 from our original News to see the similarities ion say
# Topic 9.                                                  Piruz Alemi 04/11/2020
# 
for iter_idx,piruz_idx in enumerate(Topic9[:3]):
    print('\nTopic9 material #%d' % (iter_idx + 1))
    print(dNews['Top1'][piruz_idx][:300], '...')


Topic9 material #1
mexico s failed war against the drug gangs mexicans are calling for an end to the battle against organised crime that has cost 40 000 lives ...

Topic9 material #2
pope pope benedict xvi to resign the pope is to resign at the end of this month in an entirely unexpected development reports from the vatican say  ...

Topic9 material #3
anil agarwal the billionaire founder of vedanta resources said he amp his family decided to donate 75 of their wealth to charity after meeting bill gates the worlds richest person what we earn must be returned for the greater good of society life is not only about wealth the billionaire said ...


In [62]:
# Plot
#plt.figure(figsize=(12, 12))
#plt.scatter(x, y, c=clusters)
#plt.xlabel('Component 2')
#plt.xlabel('Component 1')
#plt.title("Segregation of Topic Clusters", )

In [63]:
# Very interesting results! for some Topics this classification works fine, though for some others it does not!
# Here we observe the close topics in Topic 16

In [64]:
df = pd.read_csv('Combined_News_DJIA.csv').head()
df.iloc[-2]

Date                                            2008-08-13
Label                                                    0
Top1     b' U.S. refuses Israel weapons to attack Iran:...
Top2     b"When the president ordered to attack Tskhinv...
Top3     b' Israel clears troops who killed Reuters cam...
Top4     b'Britain\'s policy of being tough on drugs is...
Top5     b'Body of 14 year old found in trunk; Latest (...
Top6     b'China has moved 10 *million* quake survivors...
Top7     b"Bush announces Operation Get All Up In Russi...
Top8                b'Russian forces sink Georgian ships '
Top9     b"The commander of a Navy air reconnaissance s...
Top10    b"92% of CNN readers: Russia's actions in Geor...
Top11    b'USA to send fleet into Black Sea to help Geo...
Top12    b"US warns against Israeli plan to strike agai...
Top13    b"In an intriguing cyberalliance, two Estonian...
Top14    b'The CNN Effect: Georgia Schools Russia in In...
Top15         b'Why Russias response to Georgia was righ

In [65]:
# Next we define a generator function stream_docs that reads in and returns one document at a time
def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3], int(line[2])
            yield text, label

In [66]:
next(stream_docs(path='Combined_News_DJIA.csv'))


('2008-08-08,0,"b""Georgia \'downs two Russian warplanes\' as countries move to brink of war""",b\'BREAKING: Musharraf to be impeached.\',b\'Russia Today: Columns of troops roll into South Ossetia; footage from fighting (YouTube)\',"b\'Russian tanks are moving towards the capital of South Ossetia, which has reportedly been completely destroyed by Georgian artillery fire\'","b""Afghan children raped with \'impunity,\' U.N. official says - this is sick, a three year old was raped and they do nothing""",b\'150 Russian tanks have entered South Ossetia whilst Georgia shoots down two Russian jets.\',"b""Breaking: Georgia invades South Ossetia, Russia warned it would intervene on SO\'s side""","b""The \'enemy combatent\' trials are nothing but a sham: Salim Haman has been sentenced to 5 1/2 years, but will be kept longer anyway just because they feel like it.""","b\'Georgian troops retreat from S. Osettain capital, presumably leaving several hundred people killed. [VIDEO]\'",b\'Did the U.S. P

In [67]:
#dNews=pd.read_csv('Combined_News_DJIA.csv', encoding='latin-1')

In [68]:
# We will now define a function, get_minibatch, that will take a documet stream
# from the stream_docs function and return a particular number of documents
# specified by the size parameter

In [69]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
                text, label = next(doc_stream)
                docs.append(label)
    except StopIteration:
        return None, None
    return docs, y


In [70]:
# Unfortunately we cannot use CountVectorizer or TfidfVectorizer to 
# keep all the feature vectors of the training data set in memory to calculate the inverse documentary freq. 
#. So we will use a HashingVectorizer!!!! 

In [71]:
from sklearn.feature_extraction.text import HashingVectorizer

In [72]:
from sklearn.linear_model import SGDClassifier
#from sklearn.linear_model import tokenizer

In [73]:
#from keras.preprocessing.text import Tokenizer

In [74]:
#from sklearn import tokenizer

In [75]:
def tokenizer(text):
  return text.split()

In [76]:
vect = HashingVectorizer(decode_error='ignore',
                        n_features = 2**21,
                        preprocessor=None,
                        tokenizer=tokenizer)

In [77]:
# could not use n_iter = 1 , alemi 04/12/2020
#
# We reinitialize a logistic regression classifier by setting the loss parameter of SGDClassifier
# we reduce the chance of causing hash collisions, but we also increase the number of coefficients in our logistic 
# model. 
clf=SGDClassifier(loss='log', random_state=1, max_iter=1)

In [78]:
# Having set up all the complementary functions, we can now start our
#. out-of-core learning

In [79]:
import pyprind

In [80]:
pbar = pyprind.ProgBar(45)

In [81]:
classes = np.array([0,1])

In [84]:
#.  --------------------------------------------------------------------------------------------------------
#.  Here i am not sure if the X_Train needs to be vectorized again..
#
#.  We initialized the progress bar object with 45 iterations and in the following for loop
#    we iterated 45 mini batches of documents 
#.   doc_stream is a parameter in def get-minibatch(doc_stream, size), not to be confused with Stream_doc used
#    elsewhere...
#.  ---------------------------------------------------------------------------------------------------------
doc_stream=X
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

TypeError: 'csr_matrix' object is not an iterator

In [242]:
X_test, y_test = get_minibatch(doc_stream, size=1500)

NameError: name 'doc_stream' is not defined

In [122]:
X_test = vect.transform(X_test)

AttributeError: lower not found

In [123]:
print('Accuracy: %.3f' % clf.score(X_test, y_test))

NotFittedError: This SGDClassifier instance is not fitted yet

In [124]:
# Finally we use the last 1500 documents to update our model
clf = clf.partial_fit(X_test, y_test)

ValueError: classes must be passed on the first call to partial_fit.